In [1]:
%%HTML
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>

In [2]:
import torch
from lib.io import load_pickle_data
from lib.noglobal import noglobal
from sklearn.preprocessing import StandardScaler, LabelEncoder,MinMaxScaler
import torch.optim as optim

## data loader

### load data

In [3]:
train_path = "/work/data/input/selfmade_dataset/baseline_with_derived_data_v1/train.pkl"
test_path = "/work/data/input/selfmade_dataset/baseline_with_derived_data_v1/test.pkl"
df = load_pickle_data(train_path)
test_df = load_pickle_data(test_path)

train_df_path = ['2020-05-14-US-MTV-2', '2020-06-04-US-MTV-1', '2020-06-11-US-MTV-1', '2020-09-04-US-SF-2', '2021-01-04-US-RWC-2', '2021-03-10-US-SVL-1', '2021-04-28-US-MTV-1', '2021-04-29-US-SJC-2']
valid_df_path = ['2020-05-14-US-MTV-1', '2020-05-21-US-MTV-1', '2020-05-21-US-MTV-2', '2020-05-29-US-MTV-1', '2020-05-29-US-MTV-2', '2020-06-05-US-MTV-1', '2020-06-05-US-MTV-2', '2020-07-08-US-MTV-1', '2020-07-17-US-MTV-1', '2020-07-17-US-MTV-2', '2020-08-03-US-MTV-1', '2020-08-06-US-MTV-2', '2020-09-04-US-SF-1', '2021-01-04-US-RWC-1', '2021-01-05-US-SVL-1', '2021-01-05-US-SVL-2', '2021-04-15-US-MTV-1', '2021-04-22-US-SJC-1', '2021-04-26-US-SVL-1', '2021-04-28-US-SJC-1', '2021-04-29-US-MTV-1']


### 欠損値処理 (0埋め)

In [4]:
df = df.fillna(0)
test_df = test_df.fillna(0)


### ラベル

In [5]:
derrived = [["svid_correctedPrM_"+str(i),"svid_xSatPosM_"+str(i),"svid_ySatPosM_"+str(i),"svid_zSatPosM_"+str(i)] for i  in range(1,38) ]
baseline_pred = ["latDeg","lngDeg","heightAboveWgs84EllipsoidM"]
#baseline_pred = ["latDeg","lngDeg","heightAboveWgs84EllipsoidM","prev_lat","prev_lng","prev_heightAboveWgs84EllipsoidM"]


x_col =  baseline_pred +  sum( derrived, [])
y_col = ["LatDeg_gt","LngDeg_gt"]

In [6]:
len(x_col)

151

## Pytorch

https://www.kaggle.com/ebinan92/time-series-rnn-xy-prediction

### Import

In [7]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import numpy as np

### Dataset

In [8]:
class CustomDataset(Dataset):
    
    def __init__(self,df_x,df_y,transform=None,MEMORY_LENGTH=5):
        
        self.transform = transform;
        
        self.train_x = df_x.reset_index()
        self.train_y = df_y.reset_index()
        self.memory_length = MEMORY_LENGTH
        
        
        
        
    
    def __getitem__(self,idx):
        
        x = self.train_x.iloc[idx]
        y = self.train_x.iloc[idx]        
        
        x_out = torch.tensor(x.to_numpy()).float();
        y_out = torch.tensor(y.to_numpy()).float();
                        
        return x_out,y_out
        
    def __len__(self):
        return len(self.train_x);
        
    

In [9]:
class simple_MLP(nn.Module):
    
    def __init__(self,input_layer_size=152):
        super(simple_MLP,self).__init__();
        
    
        self.fc1 = nn.Linear(input_layer_size,512);
        self.fc2 = nn.Linear(512,1024);
        self.fc3 = nn.Linear(1024,512);
        self.fc4 = nn.Linear(512,128);
        self.fc5 = nn.Linear(128,2);
        
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.2)
        self.dropout4 = nn.Dropout(0.2)
        
        self.normalization1 = nn.BatchNorm1d(512)
        self.normalization2 = nn.BatchNorm1d(1024)
        self.normalization3 = nn.BatchNorm1d(512)
        self.normalization4 = nn.BatchNorm1d(128)
        
        
        
        self.relu = nn.ReLU(False);
        
        
    def forward(self, x):
        
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.normalization1(x)
        x = self.relu(x)
        
        x = self.fc2(x)
        x = self.dropout2(x)
        x = self.normalization2(x)
        x = self.relu(x)
        
        x = self.fc3(x)
        x = self.dropout3(x)
        x = self.normalization3(x)
        
        x = self.fc4(x)
        x = self.dropout4(x)
        x = self.normalization4(x)
                
        x = self.fc5(x)
        
        return x;
        

In [10]:
svid_correctedPrm = [ "svid_correctedPrM_"+str(i) for i in range(1,38)];
svid_xSatPosM = [ "svid_xSatPosM_"+str(i) for i in range(1,38)];
svid_ySatPosM = [ "svid_xSatPosM_"+str(i) for i in range(1,38)];
svid_zSatPosM = [ "svid_xSatPosM_"+str(i) for i in range(1,38)];
baseline_pred_lat = ["latDeg"]
baseline_pred_lng = ["lngDeg"]
baseline_pred_height = ["heightAboveWgs84EllipsoidM"]

## preprocessing

### svid_correctedPrm

In [11]:
svid_correctedPrm_encoder = StandardScaler()
svid_correctedPrm_encoder.fit(df.loc[:,svid_correctedPrm])

df.loc[:,svid_correctedPrm] = svid_correctedPrm_encoder.transform(df.loc[:,svid_correctedPrm]);
df.loc[:,svid_correctedPrm] = svid_correctedPrm_encoder.transform(df.loc[:,svid_correctedPrm]);


### svid_xSatPosM

In [12]:
svid_xSatPosM_encoder = StandardScaler()
svid_xSatPosM_encoder.fit(df.loc[:,svid_xSatPosM])

df.loc[:,svid_xSatPosM] = svid_xSatPosM_encoder.transform(df.loc[:,svid_xSatPosM]);
test_df.loc[:,svid_xSatPosM] = svid_xSatPosM_encoder.transform(test_df.loc[:,svid_xSatPosM]);

### svid_ySatPosM

In [13]:
svid_ySatPosM_encoder = StandardScaler()
svid_ySatPosM_encoder.fit(df.loc[:,svid_ySatPosM])

df.loc[:,svid_ySatPosM] = svid_ySatPosM_encoder.transform(df.loc[:,svid_ySatPosM]);
test_df.loc[:,svid_ySatPosM] = svid_ySatPosM_encoder.transform(test_df.loc[:,svid_ySatPosM]);

### svid_zSatPosM

In [14]:
svid_zSatPosM_encoder = StandardScaler()
svid_zSatPosM_encoder.fit(df.loc[:,svid_zSatPosM])

df.loc[:,svid_zSatPosM] = svid_zSatPosM_encoder.transform(df.loc[:,svid_zSatPosM]);
test_df.loc[:,svid_zSatPosM] = svid_zSatPosM_encoder.transform(test_df.loc[:,svid_zSatPosM]);

### heightAboveWgs84EllipsoidM

In [15]:
baseline_pred_height_encoder = StandardScaler()
baseline_pred_height_encoder.fit(df.loc[:,baseline_pred_height])

df.loc[:,baseline_pred_height] = baseline_pred_height_encoder.transform(df.loc[:,baseline_pred_height]);
test_df.loc[:,baseline_pred_height] = baseline_pred_height_encoder.transform(test_df.loc[:,baseline_pred_height]);

### baseline_pred_lat

In [16]:
baseline_pred_lat_encoder = MinMaxScaler()
baseline_pred_lat_encoder.fit(df.loc[:,baseline_pred_lat])

df.loc[:,baseline_pred_lat] = baseline_pred_lat_encoder.transform(df.loc[:,baseline_pred_lat]);
test_df.loc[:,baseline_pred_lat] = baseline_pred_lat_encoder.transform(test_df.loc[:,baseline_pred_lat]);

### baseline_pred_lat

In [17]:
baseline_pred_lng_encoder = MinMaxScaler()
baseline_pred_lng_encoder.fit(df.loc[:,baseline_pred_lng])

df.loc[:,baseline_pred_lng] = baseline_pred_lng_encoder.transform(df.loc[:,baseline_pred_lng]);
test_df.loc[:,baseline_pred_lng] = baseline_pred_lng_encoder.transform(test_df.loc[:,baseline_pred_lng]);

## Evaluation function

In [18]:

def calc_haversine(predict, output):
    
    
    lat1 = predict[:,0] + 36.0
    lon1 = predict[:,1] - 123.0
    
    
    lat2 = output[:,0] + 36.0
    lon2 = output[:,1] - 123.0
                
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(torch.rad2deg, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = torch.sin(dlat/2)**2 + torch.cos(lat1) * torch.cos(lat2) * torch.sin(dlon/2)**2
    
    a[a > 1] = 1
    a[a < 0] = 0
    
    
    dist = 2 * RADIUS * torch.arcsin(a**0.5)
    
    
    return dist.sum()


def compute_dist(oof, gt_df=None):
        
    if (gt_df is None):
        FILES = glob.glob(f"/work/data/input/google-smartphone-decimeter-challenge/train/*/*/ground_truth.csv")
        gt_list = [ pd.read_csv(f) for f in FILES ];
        gt_df = pd.concat(gt_list,axis=0);
            
    
    gt_df["phone"] = gt_df["collectionName"] + "_" + gt_df["phoneName"]
        
    df = oof.merge(gt_df, on = ['phone','millisSinceGpsEpoch'])
         
    dst_oof = calc_haversine(df.latDeg_x,df.lngDeg_x, df.latDeg_y, df.lngDeg_y)
    
    scores = pd.DataFrame({'phone': df.phone,'dst': dst_oof})
    scores_grp = scores.groupby('phone')
    
    d50 = scores_grp.quantile(.50).reset_index()
    d50.columns = ['phone','q50']
    d95 = scores_grp.quantile(.95).reset_index()
    d95.columns = ['phone','q95']
    
    return (scores_grp.quantile(.50).mean() + scores_grp.quantile(.95).mean())/2, d50.merge(d95)


In [19]:
def comp_metric_xy_(output, y):
    return torch.sqrt(((norms*(output - y[:,:2]))**2).sum())



def comp_metric_xy(output, y, norms):
    return torch.sqrt(((norms*(output - y[:,:2]))**2).sum())
def weighted_mse_loss(output, y):
    return (y[:, 2].reshape(-1,1)*(output - y[:,:2])**2).sum()
def comp_metric_weighted_xy(output, y, norms):
    return torch.sqrt(((y[:, 2].reshape(-1,1)*(norms * (output - y[:,:2]))**2)).sum())

## Traning

In [20]:
x_col =  baseline_pred +  sum( derrived, [])
y_col = ["latDeg_gt","lngDeg_gt"]




train_df = df[df.phone.str.contains("|".join(train_df_path))].reset_index()
valid_df = df[df.phone.str.contains("|".join(valid_df_path))].reset_index()


train_df_x = train_df[x_col]
train_df_y = valid_df[y_col]

valid_df_x = valid_df[x_col]
valid_df_y = valid_df[y_col]


training_loader = DataLoader(CustomDataset(train_df_x,train_df_y),batch_size=256,shuffle=False);
validation_loader = DataLoader(CustomDataset(valid_df_x,valid_df_y),batch_size=256,shuffle=False);

model = simple_MLP()

optimizer = optim.Adam(model.parameters(), lr=0.005)

criterion = comp_metric_xy_

norm_x = np.max(df.loc[:, 'latDeg_gt'].values) - np.min(df.loc[:, 'latDeg_gt'].values)
norm_y = np.max(df.loc[:, 'lngDeg_gt'].values) - np.min(df.loc[:, 'lngDeg_gt'].values)
norms = torch.tensor([[norm_x, norm_y]])


BATCH_SIZE = 256




In [21]:
@noglobal()
def train_manytomany(t_loader, v_loader, optimizer, criterion, model,norms):
    
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.6)
    model.train()
    
    early_stopping = 0    
    
    
    
    loss_pred = 10000000000000
    EPOCH = 1000
    DEVICE = "cpu"
    
    
    for e in range(EPOCH):        
        for x,y in t_loader:
            optimizer.zero_grad();
            x, y = x.to(DEVICE).float(), y.to(DEVICE).float()
            
            output = model(x)
            
            loss = criterion(output, y)
            loss.backward()
            optimizer.step();
        
        scheduler.step();
        
        if (e+1) % 1 == 0:
            train_loss = [];
            train_size = [];
            
            valid_loss = [];
            valid_sizees = [];
            
            total_point = 0
            
            with torch.no_grad():
                ## train data
                
                for (x,y) in t_loader:
                    x, y = x.to(DEVICE).float(), y.to(DEVICE).float()
                    output = model(x);                                                 
                    loss = criterion(output, y);
                    
                    train_loss.append(loss.item());                                                
                    train_size.append(output.numel());
                
                
                for (x, y) in v_loader:
                    x, y = x.to(DEVICE).float(), y.to(DEVICE).float()
                    output = model(x)
                    
                    
                    loss = criterion(output, y)
                    #loss2 = comp_metric_weighted_xy(output, y, norms)
                    
                    valid_loss.append(loss.item())
                    #losses2.append(loss2.item())
                    valid_sizees.append(output.numel())
                    
                    
                    
                
                training_score = np.sum(train_loss)/np.sum(train_size)
                validation_score = np.sum(valid_loss)/np.sum(valid_sizees)
                
                #tmp2 = np.sum(losses2)/np.sum(sizees)
                                
                print(f"epoch: {e+1}, t_loss: {training_score:.5f}, validation loss: {validation_score:.5f}, lr:{scheduler.get_last_lr()[0]:.5f}")
                    
                if loss_pred < validation_score:
                    early_stopping += 1
                else:
                    early_stopping = 0
                    loss_pred = validation_score
                    model_pred = model
                if early_stopping > 5:                        
                    return model_pred, loss_pred    
    return model_pred, loss_pred
                    
                    

In [22]:
model, loss = train_manytomany(
        training_loader, validation_loader, optimizer, criterion, model, norms)

epoch: 0, t_loss: 221.37942, validation loss: 530.97418, lr:0.00500
epoch: 1, t_loss: 215.06283, validation loss: 524.58076, lr:0.00500
epoch: 2, t_loss: 205.21708, validation loss: 514.38051, lr:0.00500
epoch: 3, t_loss: 193.27569, validation loss: 501.56067, lr:0.00500
epoch: 4, t_loss: 180.68662, validation loss: 487.39643, lr:0.00500
epoch: 5, t_loss: 168.51148, validation loss: 472.88756, lr:0.00500
epoch: 6, t_loss: 157.43551, validation loss: 458.76395, lr:0.00500
epoch: 7, t_loss: 147.86898, validation loss: 445.59022, lr:0.00500
epoch: 8, t_loss: 139.85800, validation loss: 433.57229, lr:0.00500
epoch: 9, t_loss: 133.30712, validation loss: 422.77380, lr:0.00300
epoch: 10, t_loss: 130.18164, validation loss: 417.19168, lr:0.00300
epoch: 11, t_loss: 127.52192, validation loss: 412.14769, lr:0.00300
epoch: 12, t_loss: 125.23185, validation loss: 407.53324, lr:0.00300
epoch: 13, t_loss: 123.26107, validation loss: 403.30740, lr:0.00300
epoch: 14, t_loss: 121.56699, validation los

epoch: 119, t_loss: 112.04751, validation loss: 364.93227, lr:0.00001
epoch: 120, t_loss: 112.04732, validation loss: 364.93008, lr:0.00001
epoch: 121, t_loss: 112.04711, validation loss: 364.92801, lr:0.00001
epoch: 122, t_loss: 112.04686, validation loss: 364.92576, lr:0.00001
epoch: 123, t_loss: 112.04667, validation loss: 364.92371, lr:0.00001
epoch: 124, t_loss: 112.04641, validation loss: 364.92149, lr:0.00001
epoch: 125, t_loss: 112.04624, validation loss: 364.91944, lr:0.00001
epoch: 126, t_loss: 112.04602, validation loss: 364.91724, lr:0.00001
epoch: 127, t_loss: 112.04585, validation loss: 364.91507, lr:0.00001
epoch: 128, t_loss: 112.04559, validation loss: 364.91285, lr:0.00001
epoch: 129, t_loss: 112.04544, validation loss: 364.91063, lr:0.00001
epoch: 130, t_loss: 112.04516, validation loss: 364.90943, lr:0.00001
epoch: 131, t_loss: 112.04511, validation loss: 364.90798, lr:0.00001
epoch: 132, t_loss: 112.04504, validation loss: 364.90656, lr:0.00001
epoch: 133, t_loss: 

In [23]:
print(loss)

364.8789619745306
